In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import scrapy
import os
from time import sleep
import random
from datetime import date

# Tasca M10 T01

### - Exercici 1
Realitza web scraping de dues de les tres pàgines web proposades utilitzant BeautifulSoup primer i Selenium després. 

- http://quotes.toscrape.com

- https://www.bolsamadrid.es

- www.wikipedia.es (fes alguna cerca primer i escrapeja algun contingut)

Primero vamos a realizar una request de la página y luego vamos a parsear el contenido con el parseador de html usando BeautifulSoup

In [2]:
url = "https://quotes.toscrape.com/"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

Con la herramienta de inspección vemos que todas las citas están contenidas en contenedores div que pertenecen a la clase 'quote'. Vamos a seleccionarlas.

In [3]:
quotes=soup.find_all("div", class_="quote")

In [4]:
for quote in quotes:
    print(quote, end="\n"*2)

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
<span>by <small class="author" itemprop="author">J.K. 

Ya tenemos las citas recogidas en la variable quotes, pero con mucho html y la información mezclada. Vamos a sacar ahora la información que necesitamos: el texto de la cita, el autor y los tags.

In [5]:
for quote in quotes:
    text=quote.find("span", class_="text").text
    author=quote.find("small", class_="author").text
    tag_html=quote.find_all("a", class_="tag")
    tag_list=[]
    for tag in tag_html:
        tag_list.append(tag.text)
    print(text, author, tag_list)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” Albert Einstein ['change', 'deep-thoughts', 'thinking', 'world']
“It is our choices, Harry, that show what we truly are, far more than our abilities.” J.K. Rowling ['abilities', 'choices']
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” Albert Einstein ['inspirational', 'life', 'live', 'miracle', 'miracles']
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” Jane Austen ['aliteracy', 'books', 'classic', 'humor']
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” Marilyn Monroe ['be-yourself', 'inspirational']
“Try not to become a man of success. Rather become a man of value.” Albert Einstein ['adulthood', 'success', 'value']
“It is better to be hated for what you are than to b

Crearemos un dataframe y guardaremos los datos en él

In [6]:
df = pd.DataFrame(columns=['text', 'author', 'tags'])

In [7]:
for quote in quotes:
    text=quote.find("span", class_="text").text
    author=quote.find("small", class_="author").text
    tag_html=quote.find_all("a", class_="tag")
    tag_list=[]
    for tag in tag_html:
        tag_list.append(tag.text)
    df = pd.concat([df, pd.DataFrame([{'text': text, 'author':author, 'tags': tag_list}])], ignore_index=True)

In [8]:
df.head()

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"


Como la web tiene 10 páginas y queremos almacenarlas todas, crearemos el mismo proceso que hemos hecho pero dentro de un bucle

In [9]:
df = pd.DataFrame(columns=['text', 'author', 'tags'])

In [10]:
for i in range(1,11):
    url="https://quotes.toscrape.com/page/" + str(i) + "/"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    quotes=soup.find_all("div", class_="quote")
    for quote in quotes:
        text=quote.find("span", class_="text").text
        author=quote.find("small", class_="author").text
        tag_html=quote.find_all("a", class_="tag")
        tag_list=[]
        for tag in tag_html:
            tag_list.append(tag.text)
        df = pd.concat([df, pd.DataFrame([{'text': text, 'author':author, 'tags': tag_list}])], ignore_index=True)
    

In [11]:
df.count()

text      100
author    100
tags      100
dtype: int64

La web tiene 10 citas por página, así que podemos ver que ha guardado el contenido de toda la web en el dataframe. A continuación vamos a almacenar nuestro dataframe en un csv, para poderlo usar en otro momento.

In [12]:
df.to_csv("bs_quotes.csv")

Ahora repetiremos el proceso pero con Selenium.

In [13]:
url="https://quotes.toscrape.com/"

In [14]:
opts = Options()
opts.add_argument('-headless=new')
browser = Firefox(options=opts)
browser.get(url)

In [15]:
quotes=browser.find_elements(By.CLASS_NAME, 'quote')

In [16]:
for quote in quotes:
    text=quote.find_element(By.CLASS_NAME, 'text').text
    author=quote.find_element(By.CLASS_NAME, 'author').text
    tag_html=quote.find_elements(By.CLASS_NAME, 'tags')
    tag_list=[]
    for tag in tag_html:
        tag_list.append(tag.text)
    print(text, author, tag_list)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” Albert Einstein ['Tags: change deep-thoughts thinking world']
“It is our choices, Harry, that show what we truly are, far more than our abilities.” J.K. Rowling ['Tags: abilities choices']
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” Albert Einstein ['Tags: inspirational life live miracle miracles']
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” Jane Austen ['Tags: aliteracy books classic humor']
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” Marilyn Monroe ['Tags: be-yourself inspirational']
“Try not to become a man of success. Rather become a man of value.” Albert Einstein ['Tags: adulthood success value']
“It is better to be hated for what you are than to be love

Tal como hemos hecho con Beautiful Soup ahora haremos que visite las 10 páginas que forman la web y meteremos el contenido en un dataframe

In [17]:
df = pd.DataFrame(columns=['text', 'author', 'tags'])

In [18]:
for i in range(1,11):
    url="https://quotes.toscrape.com/page/" + str(i) + "/"
    browser.get(url)
    quotes=browser.find_elements(By.CLASS_NAME, 'quote')
    for quote in quotes:
        text=quote.find_element(By.CLASS_NAME, 'text').text
        author=quote.find_element(By.CLASS_NAME, 'author').text
        tag_html=quote.find_elements(By.CLASS_NAME, 'tags')
        tag_list=[]
        for tag in tag_html:
            tag_list.append(tag.text)
        df = pd.concat([df, pd.DataFrame([{'text': text, 'author':author, 'tags': tag_list}])], ignore_index=True)
browser.quit()
    

In [19]:
df.head()

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,[Tags: change deep-thoughts thinking world]
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,[Tags: abilities choices]
2,“There are only two ways to live your life. On...,Albert Einstein,[Tags: inspirational life live miracle miracles]
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,[Tags: aliteracy books classic humor]
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,[Tags: be-yourself inspirational]


Y guardaremos los datos generados en un csv

In [20]:
df.to_csv("sel_quotes.csv")

A continuación realizaremos un scrapeo de Wikipedia, en concreto de este enlace sobre el actor escocés David Tennant https://es.wikipedia.org/wiki/David_Tennant. Queremos guardar los datos de que premios y nominaciones ha recibido a lo largo de su carrera.

Empezaremos haciendo el proceso con Beautiful Soup

In [38]:
url="https://es.wikipedia.org/wiki/David_Tennant"

In [39]:
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

Seleccionamos la tablas del artículo

In [40]:
tables=soup.find_all("table", class_="wikitable")

La tabla que buscamos es la quinta del artículo (indice 4)

In [41]:
awards_table=tables[4]

In [42]:
print(awards_table)

<table class="wikitable sortable">
<tbody><tr>
<th>Año
</th>
<th>Trabajo
</th>
<th>Premio
</th>
<th>Categoría
</th>
<th>Resultado
</th></tr>
<tr>
<td>1995</td>
<td><i>An Experienced Woman Gives Advice</i></td>
<td>Premio de Teatro Manchester Evening News<sup class="reference separada" id="cite_ref-48"><a href="#cite_note-48"><span class="corchete-llamada">[</span>48<span class="corchete-llamada">]</span></a></sup>​</td>
<td>Mejor Actor</td>
<td rowspan="" style="background-color:#fdd;color:black; text-align:center; vertical-align:middle;">Nominado
</td></tr>
<tr>
<td>1996</td>
<td><i><a href="/wiki/El_zoo_de_cristal" title="El zoo de cristal">The Glass Menagerie</a></i></td>
<td>Theatre Management Association<sup class="reference separada" id="cite_ref-49"><a href="#cite_note-49"><span class="corchete-llamada">[</span>49<span class="corchete-llamada">]</span></a></sup>​</td>
<td>Mejor Actor</td>
<td rowspan="" style="background-color:#fdd;color:black; text-align:center; vertical-align:

Ya hemos accedido a la tabla que queremos que tiene todo el html y sus respectivos hijos de donde podremos sacar la información

In [43]:
headers=awards_table.findAll("th")
columns=[]
for header in headers:
    columns.append(header.text)
print(columns)

['Año\n', 'Trabajo\n', 'Premio\n', 'Categoría\n', 'Resultado\n']


Quitamos el salto de línea al final del nombre de las columnas

In [44]:
for i,column in enumerate(columns):
    columns[i]=column[:-1]
print(columns)

['Año', 'Trabajo', 'Premio', 'Categoría', 'Resultado']


Creamos el dataframe con las columnas

In [80]:
df=pd.DataFrame(columns=columns)

Cogeremos las filas de la tabla para el contenido

In [46]:
rows=awards_table.findAll("tr")

In [47]:
rows[0]

<tr>
<th>Año
</th>
<th>Trabajo
</th>
<th>Premio
</th>
<th>Categoría
</th>
<th>Resultado
</th></tr>

Eliminaremos la primera fila, que contiene los encabezados

In [48]:
rows=rows[1:]

In [32]:
rows[0]

<tr>
<td>1995</td>
<td><i>An Experienced Woman Gives Advice</i></td>
<td>Premio de Teatro Manchester Evening News<sup class="reference separada" id="cite_ref-48"><a href="#cite_note-48"><span class="corchete-llamada">[</span>48<span class="corchete-llamada">]</span></a></sup>​</td>
<td>Mejor Actor</td>
<td rowspan="" style="background-color:#fdd;color:black; text-align:center; vertical-align:middle;">Nominado
</td></tr>

Recorreremos las filas con sus respectivas celdas e iremos llenando el dataframe

In [81]:
for row in rows:
    cells=row.findAll("td")
    year=cells[0].text
    work=cells[1].text
    award=cells[2].text
    category=cells[3].text
    result=cells[4].text
    df=pd.concat([df, pd.DataFrame([{'Año': year, 'Trabajo' : work, 'Premio' : award, 'Categoría': category, 'Resultado' : result}])])
    

In [82]:
df.head()

,Año,Trabajo,Premio,Categoría,Resultado
0,1995,An Experienced Woman Gives Advice,Premio de Teatro Manchester Evening News[48]​,Mejor Actor,Nominado\n
0,1996,The Glass Menagerie,Theatre Management Association[49]​,Mejor Actor,Nominado\n
0,2000,The Comedy of Errors,Premio Ian Charleson[50]​,Best Classical Actor Under 30,Nominado\n
0,2003,Lobby Hero,Premio Olivier[51]​,Mejor Actor,Nominado\n
0,2005,Look Back in Anger,Critics' Awards for Theatre in Scotland[52]​,Best Male Performance,Ganador\n


Eliminamos el número de nota al pie al final de la columna premio y el salto de línea al final de la columna resultado

In [83]:
df['Premio']=df['Premio'].str.extract('^(.*?)(?=\[)')
df['Resultado']=df['Resultado'].str.extract('^(\w+)')

In [84]:
df.head()

,Año,Trabajo,Premio,Categoría,Resultado
0,1995,An Experienced Woman Gives Advice,Premio de Teatro Manchester Evening News,Mejor Actor,Nominado
0,1996,The Glass Menagerie,Theatre Management Association,Mejor Actor,Nominado
0,2000,The Comedy of Errors,Premio Ian Charleson,Best Classical Actor Under 30,Nominado
0,2003,Lobby Hero,Premio Olivier,Mejor Actor,Nominado
0,2005,Look Back in Anger,Critics' Awards for Theatre in Scotland,Best Male Performance,Ganador


Ya tenemos nuestro dataframe listo para ser almacenado en un csv

In [85]:
df.to_csv("bs_david_tennant.csv")

A continuación repetiremos el proceso con Selenium. 

Primero abrimos el navegador con la url del artículo de wikipedia sobre David Tennant

In [98]:
url="https://es.wikipedia.org/wiki/David_Tennant"
opts = Options()
opts.add_argument('-headless=new')
browser = Firefox(options=opts)
browser.get(url)

Encontramos primero todas las tablas y luego en concreto la de premios.

In [99]:
tables=browser.find_elements(By.CLASS_NAME, 'wikitable')

In [100]:
awards_table=tables[4]

Leemos los cabeceros de la tabla y los usamos como columnas para el dataframe.

In [101]:
headers=awards_table.find_elements(By.TAG_NAME, "th")
columns=[]
for header in headers:
    columns.append(header.text)
print(columns)

['Año', 'Trabajo', 'Premio', 'Categoría', 'Resultado']


In [102]:
df=pd.DataFrame(columns=columns)

Localizamos todas las filas de la tabla y eliminamos la primera (cabecera)

In [103]:
rows=awards_table.find_elements(By.TAG_NAME, "tr")
rows=rows[1:]

Recorremos las filas y llenamos el dataframe y cerramos el navegador.

In [106]:
for row in rows:
    cells=row.find_elements(By.TAG_NAME, "td")
    year=cells[0].text
    work=cells[1].text
    award=cells[2].text
    category=cells[3].text
    result=cells[4].text
    df=pd.concat([df, pd.DataFrame([{'Año': year, 'Trabajo' : work, 'Premio' : award, 'Categoría': category, 'Resultado' : result}])])
browser.quit()   

In [107]:
df.head()

,Año,Trabajo,Premio,Categoría,Resultado
0,1995,An Experienced Woman Gives Advice,Premio de Teatro Manchester Evening News48,Mejor Actor,Nominado
0,1996,The Glass Menagerie,Theatre Management Association49,Mejor Actor,Nominado
0,2000,The Comedy of Errors,Premio Ian Charleson50,Best Classical Actor Under 30,Nominado
0,2003,Lobby Hero,Premio Olivier51,Mejor Actor,Nominado
0,2005,Look Back in Anger,Critics' Awards for Theatre in Scotland52,Best Male Performance,Ganador


Aquí selenium ha quitado las llaves de las notas en la columna 'Premio', pero ha dejado el número. Vamos a quitarlo

In [108]:
df['Premio']=df['Premio'].str.extract('^(.*?)(?=[0-9])')

In [109]:
df.head()

,Año,Trabajo,Premio,Categoría,Resultado
0,1995,An Experienced Woman Gives Advice,Premio de Teatro Manchester Evening News,Mejor Actor,Nominado
0,1996,The Glass Menagerie,Theatre Management Association,Mejor Actor,Nominado
0,2000,The Comedy of Errors,Premio Ian Charleson,Best Classical Actor Under 30,Nominado
0,2003,Lobby Hero,Premio Olivier,Mejor Actor,Nominado
0,2005,Look Back in Anger,Critics' Awards for Theatre in Scotland,Best Male Performance,Ganador


Ya tenemos los datos listos para ser almacenados.

In [110]:
df.to_csv("sel_david_tennant.csv")

### - Exercici 2
Documenta en un Word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.


Ver documento 'Sobre el dataset' disponible en .odt y .pdf

### - Exercici 3
Tria una pàgina web que tu vulguis i realitza web scraping mitjançant la llibreria Selenium primer i Scrapy després. 

La web elegida será https://www.autoscout24.es/ ya que mi proyecto tratará sobre el mercado de segunda mano de coches usados.

Usaremos sleep entre pasos para simular comportamiento humano

In [15]:
url="https://www.autoscout24.es/"
opts = Options()
opts.add_argument('-headless=new')
browser = Firefox(options=opts)
browser.get(url)
sleep(random.randint(1, 5))

Aceptaremos las cookies si es necesario.

In [16]:
accept_cookies_button=browser.find_element(By.CLASS_NAME, '_consent-accept_1i5cd_111')
if accept_cookies_button:
    accept_cookies_button.click()
sleep(random.randint(1, 5))

Hacemos click en el enlace de búsqueda, en principio sin indicarle parámetros de marca-modelo o ubicación.

In [17]:
search_link=browser.find_element(By.ID, "search-mask-search-cta")
search_link.click()
sleep(random.randint(2, 6))

Comprobaremos si el archivo de scrapeo ya existe y lo cargaremos para actualizarlo o si es la primera vez que se hace el scrapeo se crearán los dataframes con las columnas que necesitan antes de entrar a la página de detalle, el resto las creará con los concat. 

Se ha decidido almacenar las descripciones en un dataset distinto ya que en principio no serán importantes para el análisis y la predicción de precio. Pero podrían llegar a ser procesadas en otro momento, almacenándolas junto al enlace y el número de oferta nos permite seguirlas teniendo enlazadas a los anuncios pero sin que entorpezcan el EDA y Regresión que se aplicarán en el proyecto al dataset de ofertas.

In [18]:
fname="vehiculos_usados.csv"
fdescriptions="descripciones_vehiculos.csv"

if os.path.isfile(fname):
    df=pd.read_csv(fname)
else:
    cols=['Fecha extrac', 'Enlace']
    df=pd.DataFrame(columns=cols)

if os.path.isfile(fdescriptions):
    df_descriptions=pd.read_csv(fdescriptions)
else:
    cols=['Num oferta', 'Descripción']
    df_descriptions=pd.DataFrame(columns=cols)

In [19]:
df.count()

Unnamed: 0.11                1051
Unnamed: 0.10                1021
Unnamed: 0.9                  984
Unnamed: 0.8                  908
Unnamed: 0.7                  797
Unnamed: 0.6                  689
Unnamed: 0.5                  636
Unnamed: 0.4                  523
Unnamed: 0.3                  488
Unnamed: 0.2                  391
Unnamed: 0.1                  293
Unnamed: 0                    263
Fecha extrac                 1051
Enlace                       1051
Marca                        1051
Modelo                       1049
Precio                       1051
Localización                 1051
Potencia                     1051
Tipo vendedor                1051
Categoría                    1051
Tipo de vehículo             1051
puertas                      1051
Versión del país             1051
Núm. de oferta               1051
Garantía                      965
Kilometraje                  1051
Año                          1051
Tipo de cambio               1050
Capacidad     

Recorreremos los anuncios y aquellos que no figuren en el dataset los abriremos para poder obtener la información detallada e introducirla en el dataset. En este notebook lo limitaremos a 30 anuncios por el tiempo de ejecución. En el del proyecto irá todo en un .py en lugar de por celdas para poder ser ejecutado todo de manera automática y se sacarán 300 anuncios por ejecución.

In [20]:
anuncios=0
while anuncios < 30:
    nuevo=0
    #Se espera a que la página esté cargada esperando a que cargue un elementeo en concreto. Si no carga lanza una excepción
    try:
        element_present = EC.presence_of_element_located((By.CLASS_NAME, "SaveSearchButton_buttonChildren__o_r9y"))
        WebDriverWait(browser, 30).until(element_present)
    except TimeoutException:
        print ("Timed out waiting for page to load")
        
    listings=browser.find_elements(By.CLASS_NAME, "ListItem_wrapper__J_a_C")
    for listing in listings:
        link=listing.find_element(By.TAG_NAME, "a")
        link_address=link.get_attribute("href")
        if link_address not in df['Enlace'].values:
            #A veces petaba aquí porque link era none (por algún recuadro de publicidad, etc). Le hacemos que busque el siguiente anuncio
            try:
                link.click()
            except:
                continue
            #Se espera a que la página esté cargada eligiendo un elemento por el cual esperar. Si no carga lanza una excepción
            try:
                element_present = EC.presence_of_element_located((By.ID, "lead-form-lightbox-desktop-button"))
                WebDriverWait(browser, 30).until(element_present)
            except TimeoutException:
                print ("Timed out waiting for page to load")
            
            atributos=browser.find_elements(By.CLASS_NAME, "VehicleOverview_itemText__V1yKT")
            #A veces no está dentro de un anuncio y los atributos petaban
            try:
                dicc_vehic={
                    'Fecha extrac' : date.today(),
                    'Enlace' : link_address,
                    'Marca' : browser.find_element(By.CLASS_NAME, "StageTitle_boldClassifiedInfo__L7JmO").text,
                    'Modelo' : browser.find_element(By.CLASS_NAME, "StageTitle_model__pG_6i").text,
                    'Precio' : browser.find_element(By.CLASS_NAME, "PriceInfo_price__JPzpT").text,
                    'Localización': browser.find_element(By.CLASS_NAME, "LocationWithPin_locationItem__pHhCa").get_attribute("href"),
                    'Potencia' : atributos[4].text,
                    'Tipo vendedor' : atributos[5].text,    
                }
            except:
                continue
            datos_columnas=browser.find_elements(By.CLASS_NAME, "DataGrid_defaultDtStyle__yzRR_")
            datos_valores=browser.find_elements(By.CLASS_NAME, "DataGrid_defaultDdStyle__29SKf")
            for i,columna in enumerate(datos_columnas):
                dicc_vehic[columna.text]=datos_valores[i].text
            df_new=pd.DataFrame(dicc_vehic, index=[0])
            df=pd.concat([df, df_new])
            #Algunos anuncios daban error por no tener descripción, lo gestionamos
            try:
                description=browser.find_element(By.CLASS_NAME, "SellerNotesSection_content__S5suY").text
            except:
                description=""
            df_descriptions=pd.concat([df_descriptions, pd.DataFrame([{'Núm. de oferta': dicc_vehic['Núm. de oferta'], 'Enlace': link_address, 'Descripción' : description}])])
            anuncios+=1
            nuevo=1
            #Se simula que un humano está leyendo el anuncio con un sleep random entre 5 y 10 segundos
            sleep(random.randint(5, 10))
            browser.back()
            sleep(random.randint(5, 10))
            #Al volver a la página anterior se han perdido algunas variables, así que se reinicia el bucle
            break
    #Si ha llegado al final de los listings sin añadir ninguno nuevo se cambia de página (para evitar que cambie de pag con el break de almacenar anuncio)
    #Se espera a que cargue el link para hacer click para que no crashee
    if not nuevo: 
        try:
            element_present = EC.presence_of_element_located((By.CLASS_NAME, "FilteredListPagination_button__41hHM"))
            WebDriverWait(browser, 30).until(element_present)
            pag_siguiente=browser.find_elements(By.CLASS_NAME, "FilteredListPagination_button__41hHM")[-1]
            pag_siguiente.click()
            sleep(random.randint(5, 10))
        except TimeoutException:
            print ("Timed out waiting for page to load")
            continue
browser.quit()

Con el count del dataframe, como teníamos el previo podemos ver si se ha añadido la cantidad correcta de anuncios.

In [21]:
df.count()

Unnamed: 0.11                1051
Unnamed: 0.10                1021
Unnamed: 0.9                  984
Unnamed: 0.8                  908
Unnamed: 0.7                  797
Unnamed: 0.6                  689
Unnamed: 0.5                  636
Unnamed: 0.4                  523
Unnamed: 0.3                  488
Unnamed: 0.2                  391
Unnamed: 0.1                  293
Unnamed: 0                    263
Fecha extrac                 1081
Enlace                       1081
Marca                        1081
Modelo                       1079
Precio                       1081
Localización                 1081
Potencia                     1081
Tipo vendedor                1081
Categoría                    1081
Tipo de vehículo             1081
puertas                      1081
Versión del país             1081
Núm. de oferta               1081
Garantía                      988
Kilometraje                  1081
Año                          1081
Tipo de cambio               1080
Capacidad     

In [22]:
df_descriptions.count()

Unnamed: 0.11     1051
Unnamed: 0.10     1021
Unnamed: 0.9       984
Unnamed: 0.8       908
Unnamed: 0.7       797
Unnamed: 0.6       689
Unnamed: 0.5       636
Unnamed: 0.4       523
Unnamed: 0.3       488
Unnamed: 0.2       391
Unnamed: 0.1       293
Unnamed: 0         263
Num oferta           0
Descripción       1077
Núm. de oferta    1081
Enlace            1081
dtype: int64

Guardamos los dataframes en archivos csv. Quedaría trabajo de limpieza de los datos, pero como el scrapeo está pensado para ser un proceso que se repita muchas veces, se volverían a generar "datos sucios". Así que la limpieza del mismo se realizará durante el proyecto, una vez hayan realizados todas las repeticiones de scrapeo necesarias.

In [23]:
df.to_csv("vehiculos_usados.csv")
df_descriptions.to_csv("descripciones_vehiculos.csv")